In [ ]:
import json
import glob
from collections import defaultdict

# Load all json files
transaction_types = ['deposits', 'borrows', 'withdraws', 'repays', 'liquidates']
files = [
    './compoundV2_transactions_ethereum_chunk_0.json', 
    './compoundV2_transactions_ethereum_chunk_1.json', 
    './compoundV2_transactions_ethereum_chunk_2.json'
]

data = defaultdict(list)

for file in files:
    with open(file) as f:
        content = json.load(f)
        for tx_type in transaction_types:
            if tx_type in content:
                data[tx_type].extend(content[tx_type])


In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

wallet_stats = defaultdict(lambda: {
    'total_deposit': 0.0,
    'total_withdraw': 0.0,
    'total_borrow': 0.0,
    'total_repay': 0.0,
    'total_liquidated': 0.0,
    'total_liquidator_profit': 0.0,
    'num_liquidated': 0,
    'num_liquidator': 0,
    'assets': set(),
    'first_seen': float('inf'),
    'last_seen': 0,
    'tx_count': 0
})

# Helper to update time range
def update_time(stat, ts):
    ts = int(ts)
    stat['first_seen'] = min(stat['first_seen'], ts)
    stat['last_seen'] = max(stat['last_seen'], ts)

# Process non-liquidation transactions
for tx_type in ['deposits', 'withdraws', 'borrows', 'repays']:
    for tx in data[tx_type]:
        wallet = tx['account']['id']
        usd_amt = float(tx.get('amountUSD', 0))
        asset_symbol = tx['asset']['symbol']
        timestamp = tx['timestamp']

        stat = wallet_stats[wallet]
        stat[f'total_{tx_type[:-1]}'] += usd_amt  # remove trailing 's'
        stat['assets'].add(asset_symbol)
        update_time(stat, timestamp)
        stat['tx_count'] += 1

# Process liquidations
for tx in data['liquidates']:
    liquidatee = tx['liquidatee']['id']
    liquidator = tx['liquidator']['id']
    usd_amt = float(tx.get('amountUSD', 0))
    asset_symbol = tx['asset']['symbol']
    timestamp = tx['timestamp']

    wallet_stats[liquidatee]['total_liquidated'] += usd_amt
    wallet_stats[liquidatee]['num_liquidated'] += 1
    update_time(wallet_stats[liquidatee], timestamp)

    wallet_stats[liquidator]['total_liquidator_profit'] += usd_amt
    wallet_stats[liquidator]['num_liquidator'] += 1
    update_time(wallet_stats[liquidator], timestamp)

    wallet_stats[liquidator]['assets'].add(asset_symbol)
    wallet_stats[liquidator]['tx_count'] += 1

In [3]:
rows = []

for wallet, stats in wallet_stats.items():
    duration_days = (stats['last_seen'] - stats['first_seen']) / 86400 if stats['first_seen'] != float('inf') else 0
    rows.append({
        'wallet': wallet,
        'tx_count': stats['tx_count'],
        'net_deposit': stats['total_deposit'] - stats['total_withdraw'],
        'net_borrow': stats['total_borrow'] - stats['total_repay'],
        'total_liquidated': stats['total_liquidated'],
        'liquidator_profit': stats['total_liquidator_profit'],
        'num_liquidated': stats['num_liquidated'],
        'num_liquidator': stats['num_liquidator'],
        'asset_diversity': len(stats['assets']),
        'active_days': duration_days
    })

df = pd.DataFrame(rows)

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

features = df.drop(columns=['wallet'])
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

pca = PCA(n_components=1)
principal_score = pca.fit_transform(scaled_features)

# Normalize score between 0 and 100
score_scaled = MinMaxScaler((0, 100)).fit_transform(principal_score)

df['score'] = score_scaled.round(2)

In [6]:
wallet_scores = df[['wallet', 'score']].sort_values(by='score', ascending=False)

top_1000 = wallet_scores.sort_values(by='score', ascending=False).head(1000)
top_1000.to_csv("top_1000_wallet_scores.csv", index=False)

In [7]:
# Get top 5 and bottom 5 wallets by score
top_5 = df.sort_values(by='score', ascending=False).head(5)
bottom_5 = df.sort_values(by='score', ascending=True).head(5)

# Show detailed stats for them
print("\nTop 5 Wallets - Feature Details:\n", top_5)
print("\nBottom 5 Wallets - Feature Details:\n", bottom_5)


Top 5 Wallets - Feature Details:
                                           wallet  tx_count   net_deposit  \
1028  0x93667dcc21f791f71c4d8ec19ab5fcbbae06a389        14  19235.625386   
441   0x8376b14104dbf0642a3c2adfa33793cfcb0b8d76        12     10.874799   
774   0xfccb964c514c12794509ed62fa274f0e284cee82         5    741.867498   
2013  0xa3c6dc326ce56d83730c67d2a0b9dddfc9fe9258         2     13.393047   
2526  0x89b6aeabcbd42b56979d5fce7be21ac8382d9bd0         2     91.325995   

       net_borrow  total_liquidated  liquidator_profit  num_liquidated  \
1028  8215.791419      11475.886491                0.0              14   
441     10.249571        338.274244                0.0              15   
774    350.000000        382.387446                0.0               5   
2013     4.420440         55.243320                0.0               3   
2526    50.032739        126.165793                0.0               7   

      num_liquidator  asset_diversity  active_days   score  
10